In [1]:
import pip
def install_and_import(package):
    import importlib
    try:
        importlib.import_module(package)
    except ImportError:
        import pip
        pip.main(['install', package])
    finally:
        globals()[package] = importlib.import_module(package)

In [2]:
# Import necessary libraries
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import pprint

In [3]:
# Checking all the ones in NYSE and NASDAQ
f = open('./g2g.txt','r')
file_list = f.readlines()
G2G = ['^GSPC']
for line in file_list: 
    splited_line =line.split(':') 
    if splited_line[0] in ['NYSE', 'NASDAQ']:
        G2G.append(splited_line[1].strip())
print(G2G)

['^GSPC', 'ABT', 'KMX', 'FNMA', 'PG', 'KMB', 'KR', 'NUE', 'PM', 'PBI', 'WBA', 'WFC']


In [4]:
# Set start to end dates
start_date = '1999-01-01'
end_date = '2019-01-01'

In [5]:
appended_data = []

for ticker in G2G: 
    print(ticker)
    if ticker == 'KMX':
        continue
    stock_data = data.DataReader(ticker, 'yahoo', start_date, end_date)
    appended_data.append(stock_data['Close'])

result = pd.concat(appended_data, axis=1)    
# pprint(result)

^GSPC
ABT
KMX
FNMA
PG
KMB
KR
NUE
PM
PBI
WBA
WFC


In [7]:
# Rename Column names
G2G.remove('KMX')
print(G2G)
result.columns = G2G

ValueError: list.remove(x): x not in list

In [8]:
# Write Daily data
result.to_csv('./G2G_YH.csv')
# Write Normalized Daily data
result_norm = (result - result.mean()) / (result.max() - result.min())
result_norm.to_csv('./G2G_norm_YH.csv')

In [9]:
# Resample data to Monthly
df = pd.read_csv("./G2G_YH.csv", parse_dates =["Date"], index_col ="Date") 
Monthly_resampled_data = df.resample('M').mean()

In [10]:
# Write the Monthly data
Monthly_resampled_data.to_csv('./G2G_Monthly.csv')

In [11]:
# Write normalized Monthly data
Monthly_resampled_data_norm = (Monthly_resampled_data - Monthly_resampled_data.mean()) / (Monthly_resampled_data.max() - Monthly_resampled_data.min())
Monthly_resampled_data_norm.to_csv('./G2G_norm_Monthly.csv')

In [12]:
# Function to get the first None Empty Value
def get_first_values(x):
    if x.first_valid_index() is None:
        return None
    else:
        return x[x.first_valid_index()]

In [13]:
# Get percentage change from Monthly data
First_non_NaN = Monthly_resampled_data.apply(get_first_values)
percent_change_monthly = Monthly_resampled_data/ First_non_NaN
# pprint(percent_change_monthly)
percent_change_monthly.to_csv('./G2G_percent_change_Monthly_1999_base.csv')

In [14]:
percent_change_monthly_mtm = percent_change_monthly.pct_change()
percent_change_monthly_mtm.to_csv('./G2G_mtm_change.csv')